In [1]:
from asteroid_interp import interp

In [ ]:
interp("let v = 1. let v = 2.")

In [ ]:
array_rval = \
'''
-- reverse the list
let a = [1,2,3].
let a = a[2,1,0].
print a.

-- access multidim array
let b = [[1,2,3],
         [4,5,6],
         [7,8,9]].
let e = b[1][1]
print e.
'''
interp(array_rval, exceptions=True)

In [3]:
array_lval = \
'''
let a = [1,2,3].
let a[2],a[1],a[0] = a.
print a
'''
interp(array_lval, exceptions=True)


(seq 
  |(assign 
  |  |(id a) 
  |  |(list 
  |  |  |[ 
  |  |  |  |(integer 1) 
  |  |  |  |(integer 2) 
  |  |  |  |(integer 3)])) 
  |(seq 
  |  |(assign 
  |  |  |(list 
  |  |  |  |[ 
  |  |  |  |  |(juxta 
  |  |  |  |  |  |(id a) 
  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |[ 
  |  |  |  |  |  |  |  |  |(integer 2)]) 
  |  |  |  |  |  |  |(nil))) 
  |  |  |  |  |(juxta 
  |  |  |  |  |  |(id a) 
  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |[ 
  |  |  |  |  |  |  |  |  |(integer 1)]) 
  |  |  |  |  |  |  |(nil))) 
  |  |  |  |  |(juxta 
  |  |  |  |  |  |(id a) 
  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |[ 
  |  |  |  |  |  |  |  |  |(integer 0)]) 
  |  |  |  |  |  |  |(nil)))]) 
  |  |  |(id a)) 
  |  |(seq 
  |  |  |(print 
  |  |  |  |(id a) 
  |  |  |  |(file-name 
  |  |  |  |  |(nil))) 
  |  |  |(nil))))
assigning ('integer', '1') to [('integer', '1'), ('integer', '2'), (

In [ ]:
graphics = \
'''
-- Asteroid graphics
-- we can incorporate Python graphics into Asteroid via
-- escaped Python code

function circle with x, y, r do escape
"
#########################################################
vx = float(state.symbol_table.lookup_sym('x')[1])
vy = float(state.symbol_table.lookup_sym('y')[1])
vr = float(state.symbol_table.lookup_sym('r')[1])

import matplotlib.pyplot as plt

circle = plt.Circle((vx, vy), vr, color='blue')
fig, ax = plt.subplots()
ax.add_artist(circle)
plt.show()
#########################################################
"
end function

-- call the escaped function
circle(.5, .5, .2)
'''
interp(graphics, exceptions=False, tree_dump=False, symtab_dump=False)

In [ ]:
attach = \
'''
-- the 'add' function implements the behavior for 
-- the '+' operator for ints, reals, and strings.

function add with a, b do return escape 
"
###################################################################
# return register from 'escape' calls
global __retval__ 

# 'promote' is the type promotion table for primitive
# builtin types which implement the type hierarchy:
#    integer < real < string
type = promote(state.symbol_table.lookup_sym('a')[0], 
               state.symbol_table.lookup_sym('b')[0])

# select the correct add according to type
if type == 'integer':
    __retval__ = ('integer', 
            int(state.symbol_table.lookup_sym('a')[1]) + 
            int(state.symbol_table.lookup_sym('b')[1]))

elif type == 'real':
    __retval__ = ('real', 
            float(state.symbol_table.lookup_sym('a')[1]) + 
            float(state.symbol_table.lookup_sym('b')[1]))

elif type == 'string':
    __retval__ = ('string', 
            str(state.symbol_table.lookup_sym('a')[1])+
            str(state.symbol_table.lookup_sym('b')[1]))

else:
    raise ValueError('unsupported type in add')
###################################################################
"     
end function

-- attach the behavior to the '+' constructor
attach add to __plus__.

-- test the '+' operator with different data types
let x = 1.2 + 2 .
print x.
print "the output is: " + x.
-- print [1,2] + 3.
'''
interp(attach, tree_dump=False, symtab_dump=False, exceptions=False)

In [ ]:
interp("let x = 1 + 2. print x.")

In [ ]:
interp("let x = 1.3 . print x.")

In [ ]:
interp("let 1,y = 1,2. print 1,y.")

In [ ]:
interp("let x,2 = 1,2. print x,2.")

In [ ]:
interp("let s = 1,2. let x,y = s. print x,y.")

In [ ]:
interp("function ident with n do return n end function print ident(2).")

In [ ]:
interp("function ident with n do return n end function print ident ident 2 .", exceptions=True)

In [ ]:
interp("let _, x = [1], 2.")

In [ ]:
lambda1 = \
'''
load "std_ops.ast"
print (lambda with n do return n+1) 1
'''
interp(lambda1, tree_dump=False, symtab_dump=False)

In [ ]:
curried = \
'''
-- Asteroid does not support function currying because
-- of the dynamic scoping of functions

print (lambda with a do return lambda with b do return a + b) 1 2.
'''

interp(curried, tree_dump=False)